In [1]:
import sqlite3
from datetime import datetime
from tabulate import tabulate  # para exibição bonita em tabela
import shutil #quebra de lina automática na visualização da tabela

# Conexão com o banco de dados SQLite
conn = sqlite3.connect("holon_chat.db")
cursor = conn.cursor()

# Criação da tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS messages (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT NOT NULL,
    role TEXT CHECK(role IN ('user', 'holon')) NOT NULL,
    message TEXT NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    total_tokens_usage INTEGER
)
""")
conn.commit()

# Função para salvar mensagens
def save_message(user_id, role, message, total_tokens_usage):
    timestamp = datetime.now().isoformat()
    cursor.execute("""
        INSERT INTO messages (user_id, role, message, timestamp, total_tokens_usage)
        VALUES (?, ?, ?, ?, ?)
    """, (user_id, role, message, timestamp, total_tokens_usage))
    conn.commit()

# Exemplo de uso
save_message("user123", "user", "O que é holismo?", 38291)
save_message("user123", "holon", "Holismo é a ideia de que o todo é maior do que a soma das partes.", 890)


In [3]:


def exibir_tabela(db_name="holon_chat.db", table_name="messages"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Consulta os dados
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Cabeçalhos
    headers = [description[0] for description in cursor.description]

    # Detecta largura do terminal
    terminal_width = shutil.get_terminal_size().columns

    # Trunca campos longos para caber melhor na tela
    max_col_width = max(15, terminal_width // len(headers) - 5)

    def trunc(value):
        return str(value)[:max_col_width] + ("…" if len(str(value)) > max_col_width else "")

    rows_trunc = [[trunc(cell) for cell in row] for row in rows]

    # Exibe a tabela
    print(f"\n📋 Conteúdo da tabela '{table_name}':\n")
    print(tabulate(rows_trunc, headers=headers, tablefmt="fancy_grid"))

    conn.close()

# Uso
exibir_tabela()



📋 Conteúdo da tabela 'messages':

╒══════╤═══════════╤════════╤══════════════════╤══════════════════╤══════════════════════╕
│   id │ user_id   │ role   │ message          │ timestamp        │   total_tokens_usage │
╞══════╪═══════════╪════════╪══════════════════╪══════════════════╪══════════════════════╡
│    1 │ user123   │ user   │ O que é holismo… │ 2025-05-15T18:1… │                38291 │
├──────┼───────────┼────────┼──────────────────┼──────────────────┼──────────────────────┤
│    2 │ user123   │ holon  │ Holismo é a ide… │ 2025-05-15T18:1… │                  890 │
├──────┼───────────┼────────┼──────────────────┼──────────────────┼──────────────────────┤
│    3 │ user123   │ user   │ O que é holismo… │ 2025-05-15T18:1… │                38291 │
├──────┼───────────┼────────┼──────────────────┼──────────────────┼──────────────────────┤
│    4 │ user123   │ holon  │ Holismo é a ide… │ 2025-05-15T18:1… │                  890 │
├──────┼───────────┼────────┼──────────────────┼───────